In [11]:
import pandas as pd
import numpy as np

'LOAD HEAT WAVE DATA'
heat_wave = pd.read_csv("../out/mortality_heat_wave.csv", sep = ",")
print(heat_wave)

'LOAD DEATH DATA WITH TOTAL WEEKLY POPULATION BY ARRONDISSEMENT (AS OBTAINED BY LINEAR INTERPOLATION)'
mort_interp = pd.read_csv("../out/weekly_mortality_with_linear_interpolated_population_data.csv",
                         sep = ",")
print(mort_interp)
mort_interp.info()

       Unnamed: 0 YEAR_WEEK  NIS_Code Nom_arrondissement  \
0               0  2000-001     11000          Antwerpen   
1               1  2000-001     11000          Antwerpen   
2               2  2000-002     11000          Antwerpen   
3               3  2000-002     11000          Antwerpen   
4               4  2000-003     11000          Antwerpen   
...           ...       ...       ...                ...   
81137       81137  2019-050     93000      Philippeville   
81138       81138  2019-050     93000      Philippeville   
81139       81139  2019-051     93000      Philippeville   
81140       81140  2019-052     93000      Philippeville   
81141       81141  2019-052     93000      Philippeville   

       heatwave_week_boolean  ARRON       COD  N_MASK  
0                          0  11000  external     NaN  
1                          0  11000   natural    31.0  
2                          0  11000  external    13.0  
3                          0  11000   natural   263.0  

In [12]:
mort_interp[mort_interp['ARRON']=='21000']


,YEAR,COD,ARRON,N_DEATHS,WEEK,population_t


In [13]:
heat_wave.head()

,Unnamed: 0,YEAR_WEEK,NIS_Code,Nom_arrondissement,heatwave_week_boolean,ARRON,COD,N_MASK
0,0,2000-001,11000,Antwerpen,0,11000,external,NaN
1,1,2000-001,11000,Antwerpen,0,11000,natural,31.0
2,2,2000-002,11000,Antwerpen,0,11000,external,13.0
3,3,2000-002,11000,Antwerpen,0,11000,natural,263.0
4,4,2000-003,11000,Antwerpen,0,11000,external,11.0


In [14]:
'FILTER HEAT_WAVE DATA TO GET ONLY NATURAL DEATHS'
heat_wave = heat_wave[heat_wave['COD'] == "natural"]

'CONVERT YEAR_WEEK TO STRING'
heat_wave = heat_wave.astype({"YEAR_WEEK": str})

'CREATING YEAR AND WEEK VARIABLES'
heat_wave['YEAR'] = heat_wave['YEAR_WEEK'].astype(str).str[:4]
heat_wave['YEAR'] = heat_wave['YEAR'].astype(int)

heat_wave['WEEK'] = heat_wave['YEAR_WEEK'].astype(str).str[6:8]
heat_wave['WEEK'] = heat_wave['WEEK'].astype(int)

print(heat_wave)
heat_wave.info()

       Unnamed: 0 YEAR_WEEK  NIS_Code Nom_arrondissement  \
1               1  2000-001     11000          Antwerpen   
3               3  2000-002     11000          Antwerpen   
5               5  2000-003     11000          Antwerpen   
7               7  2000-004     11000          Antwerpen   
9               9  2000-005     11000          Antwerpen   
...           ...       ...       ...                ...   
81134       81134  2019-048     93000      Philippeville   
81136       81136  2019-049     93000      Philippeville   
81138       81138  2019-050     93000      Philippeville   
81139       81139  2019-051     93000      Philippeville   
81141       81141  2019-052     93000      Philippeville   

       heatwave_week_boolean  ARRON      COD  N_MASK  YEAR  WEEK  
1                          0  11000  natural    31.0  2000     1  
3                          0  11000  natural   263.0  2000     2  
5                          0  11000  natural   219.0  2000     3  
7          

In [15]:
heat_wave[heat_wave['ARRON']==21000]

,Unnamed: 0,YEAR_WEEK,NIS_Code,Nom_arrondissement,heatwave_week_boolean,ARRON,COD,N_MASK,YEAR,WEEK
6163,6163,2000-001,21000,Bruxelles-Capitale,0,21000,natural,43.0,2000,1
6165,6165,2000-002,21000,Bruxelles-Capitale,0,21000,natural,279.0,2000,2
6167,6167,2000-003,21000,Bruxelles-Capitale,0,21000,natural,254.0,2000,3
6169,6169,2000-004,21000,Bruxelles-Capitale,0,21000,natural,275.0,2000,4
6171,6171,2000-005,21000,Bruxelles-Capitale,0,21000,natural,231.0,2000,5
...,...,...,...,...,...,...,...,...,...,...
8233,8233,2019-048,21000,Bruxelles-Capitale,0,21000,natural,145.0,2019,48
8235,8235,2019-049,21000,Bruxelles-Capitale,0,21000,natural,137.0,2019,49
8237,8237,2019-050,21000,Bruxelles-Capitale,0,21000,natural,154.0,2019,50
8239,8239,2019-051,21000,Bruxelles-Capitale,0,21000,natural,194.0,2019,51


In [16]:
'MERGING HEAT_WAVE AND INTERPOLATION DATASETS'

merged_data = pd.merge(mort_interp, heat_wave, how = 'right', left_on=['YEAR', 'ARRON', 'WEEK'], right_on = ['YEAR', 'ARRON', 'WEEK'])
merged_data[merged_data['N_DEATHS'].isnull()& merged_data['N_MASK'].notnull()]
#no obsevation with null N_MASK/COD_y and not null N_DEATHS/COD_x, so drop N_DEATHS/COD_x
merged_data=merged_data.drop(columns=['COD_x','N_DEATHS','NIS_Code','Unnamed: 0'])
merged_data.head()
#merged_data.info()

,YEAR,ARRON,WEEK,population_t,YEAR_WEEK,Nom_arrondissement,heatwave_week_boolean,COD_y,N_MASK
0,2000,11000,1,932103.0,2000-001,Antwerpen,0,natural,31.0
1,2000,11000,2,932093.0,2000-002,Antwerpen,0,natural,263.0
2,2000,11000,3,932083.0,2000-003,Antwerpen,0,natural,219.0
3,2000,11000,4,932073.0,2000-004,Antwerpen,0,natural,226.0
4,2000,11000,5,932063.0,2000-005,Antwerpen,0,natural,229.0


In [17]:
'SAVING MERGED DATA TO OUT FILE'
merged_data.to_csv('../out/merged.csv', encoding = 'utf-8-sig') 